In [1]:
import torch
import torch.nn as nn

import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join('..')))

from models._model import CBPLTrainer

In [2]:
config = {
    "peak_regions":"/gladstone/corces/lab/users/vishvak/chrombpnet_tutorial/own_data/test.chr1.adjusted.bed",
    "nonpeak_regions":"/gladstone/corces/lab/users/vishvak/chrombpnet_tutorial/own_data/test.chr1.negatives.adjusted.bed",
    "genome_fasta":"/gladstone/corces/lab/users/vishvak/chrombpnet_tutorial/own_data/chr1.fa",
    "cts_bw_file":"/gladstone/corces/lab/users/vishvak/chrombpnet_tutorial/own_data/test.chr1.bw",
    "negative_sampling_ratio":8,
    "train_size": 0.7,
    "batch_size": 32,
    "filters": 64,
    "n_dil_layers": 3,
    "conv1_kernel_size": 7,
    "dilation_kernel_size" : 3,
    "profile_kernel_size": 5,
    "num_tasks": 1,
    "sequence_len": 1000,
    "out_pred_len": 1000,
    "learning_rate": 0.001,
}

In [3]:
trainer =  CBPLTrainer(config)

Read in bed file of 6013 peaks
Read in bed file of 16900 peaks
Peak length = 1000    6013
Name: count, dtype: int64
Peak length = 1000    16900
Name: count, dtype: int64
Successfully loaded in data with 6013 positive and 16900 nonpeak regions!


In [4]:
trainer.dataset.seqs.shape

torch.Size([22913, 4, 1000])

In [5]:
for batch in trainer.train_dataloader:
    inputs, targets = batch
    break  

In [6]:
inputs.shape

torch.Size([32, 4, 1000])

In [7]:
targets.shape

torch.Size([32, 1000])

In [4]:
trainer.model

BPNetLightning(
  (mse_loss): MSELoss()
  (initial_conv): CNNModule(
    (conv): Conv1d(4, 64, kernel_size=(7,), stride=(1,), padding=(3,))
    (activation): ReLU()
  )
  (dilated_convs): ModuleList(
    (0): DilatedConvModule(
      (conv): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(2,), dilation=(2,))
      (activation): ReLU()
    )
    (1): DilatedConvModule(
      (conv): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(4,), dilation=(4,))
      (activation): ReLU()
    )
    (2): DilatedConvModule(
      (conv): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(8,), dilation=(8,))
      (activation): ReLU()
    )
  )
  (profile_conv): CNNModule(
    (conv): Conv1d(64, 1, kernel_size=(5,), stride=(1,), padding=(2,))
    (activation): ReLU()
  )
  (profile_crop): Cropping1D()
  (global_avg_pool): GlobalAvgPool1D()
  (count_dense): Linear(in_features=1000, out_features=1000, bias=True)
  (flatten): Flatten(
    (flatten): Flatten(start_dim=1, end_dim=-1)
  )
)

In [5]:
trainer.fit(max_epochs=2,batch_size=64)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
2024-10-04 15:59:22.042643: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-10-04 15:59:22.042675: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.

  | Name            | Type            | Params | Mode 
------------------------------------------------------------
0 | mse_loss        | MSELoss         | 0      | train
1 | initial_conv    | CNNModule       | 1.9 K  | train
2 | dilated_convs   | ModuleList      | 37.1 K | train
3 | profile_conv    | CNNModule       | 321    | train
4 | profile_crop    | Cropping1D      | 0      | train
5 | global_avg_pool | GlobalAvgPool1D | 0      | train
6 | count_dense     | Linear          | 1.0 M  | train

Epoch 1: 100%|██████████| 502/502 [00:20<00:00, 24.41it/s, v_num=0]        

`Trainer.fit` stopped: `max_epochs=2` reached.


Epoch 1: 100%|██████████| 502/502 [00:20<00:00, 24.35it/s, v_num=0]
